# バクテリアゲノム解析

このファイルは bash カーネルを使用しています。bash カーネルをインストールしていない場合は、ターミナルで
```
pip install bash_kernel
python -m bash_kernel.install
```
を実行した後、再度 Jupyter notebook を起動してください。

anaconda or miniconda は既にインストールされているものとします。

本ファイルのコマンドを順に実行した場合、カレントディレクトリ以下に解析作業用のフォルダが作られます。（解析教本の本文で示した手順とは異なります）

`shift-Enter` を押すと選択中のセルの内容を実行することが可能です。

##  解析ツールのインストール

### FastQCのインストール

インストールの続行確認を省略するため `-y` オプションを指定して実行しています。

In [ ]:
conda install -y -c bioconda fastqc

In [ ]:
fastqc -v

### fastpのインストール

In [ ]:
conda install -y -c bioconda fastp

動作確認のためバージョンを表示

In [ ]:
fastp -v

### seqkitのインストール

In [ ]:
conda install -y -c bioconda seqkit

動作確認のためバージョンを表示

In [ ]:
seqkit version

### Platanus_Bのインストール

カレントディレクトリに`tools`ディレクトリを作成し、インストールを行います。Platanus_Bは[開発元ウェブサイト](http://platanus.bio.titech.ac.jp/platanus-b)から取得します。

In [ ]:
mkdir tools
cd tools
curl http://platanus.bio.titech.ac.jp/?ddownload=411 --output platanus_tmp.tar.gz
tar xvfz platanus_tmp.tar.gz
cp Platanus_B_v1.1.0_190607_macOS_bin/platanus_b ./
cp Platanus_B_v1.1.0_190607_macOS_bin/README ./platanus_README
rm -rf Platanus_B_v1.1.0_190607_macOS_bin platanus_tmp.tar.gz
cd ../

tools以下にインストールできたことを確認

In [ ]:
ls -l tools/

platanus_bのバージョン番号・ヘルプが表示されることを確認する。

In [ ]:
tools/platanus_b

後の操作を簡単にするため環境変数`PATH`に`tools`ディレクトリを追加。  
（この操作をJupyter notebook以外から行う場合には、カレントディレクトリの場所によって正しく動作しない場合がありますのでご注意ください）

In [ ]:
export PATH=$PWD/tools:$PATH

再度、動作確認を行い、platanus_bのバージョン番号・ヘルプが表示されることを確認する。

In [ ]:
platanus_b

## データの入手と前処理

以下の手順では、カレントディレクトリに作業フォルダとして`analysis`という名称のディレクトリを作成してディレクトリ内で解析を行います。

In [ ]:
mkdir analysis
cd analysis
pwd

`reads`というディレクトリを作り、その中にリード配列のデータを取得します。

In [ ]:
mkdir reads
cd reads
pwd

__テスト用データの取得__

ネットワーク環境によっては数分かかります。

In [ ]:
curl -O ftp://ftp.ddbj.nig.ac.jp//ddbj_database/dra/fastq/DRA002/DRA002643/DRX022186/DRR024501_1.fastq.bz2
curl -O ftp://ftp.ddbj.nig.ac.jp//ddbj_database/dra/fastq/DRA002/DRA002643/DRX022186/DRR024501_2.fastq.bz2

データの展開

In [ ]:
bunzip2 *bz2

__取得したデータの確認__

In [ ]:
seqkit stats *fastq

__FastQCの実行__

In [ ]:
fastqc *fastq

__アダプター配列の除去__

In [ ]:
fastp -i DRR024501_1.fastq -I DRR024501_2.fastq -o DRR024501_1.fastp.fastq -O DRR024501_2.fastp.fastq

__再度FastQCを実行__

In [ ]:
fastqc *.fastp.fastq

__fastp 処理結果の確認__

In [ ]:
seqkit stats *fastq

__データのサブサンプリング__

カバレッジが約100xになるよう、全データのうち20%を抽出する。（ゲノムサイズは約2.5Mbpと想定）

In [ ]:
seqkit sample -p 0.2 DRR024501_1.fastp.fastq > DRR024501_1.sampled.fastp.fastq
seqkit sample -p 0.2 DRR024501_2.fastp.fastq > DRR024501_2.sampled.fastp.fastq

__サブサンプリングしたデータの確認__

In [ ]:
seqkit stats *.sampled.fastp.fastq

## アセンブリ

ディレクトリの準備  
`assembly`という名称で作業ディレクトリを作成する。

In [ ]:
cd ../
mkdir assembly
cd assembly
pwd

（確認）リードデータは、一つ上の階層にあるreadsディレクトリに格納されている

In [ ]:
ls -l ../reads/

__アセンブリ1 (platanus_b の assemble ステップ)__

In [ ]:
platanus_b assemble -t 2 -f ../reads/DRR024501_1.sampled.fastp.fastq ../reads/DRR024501_2.sampled.fastp.fastq

上記コマンドが正しく実行できない場合、`tools`ディレクトリに`PATH`が通っていること、および、
```
ls ../reads/DRR024501_1.sampled.fastp.fastq ../reads/DRR024501_2.sampled.fastp.fastq
```
を実行してみて`reads`ディレクトリにファイルが存在していることを確認すること。

アセンブリ結果 (out_contig.fa) の確認

In [ ]:
seqkit stats -a -G N out_contig.fa

__アセンブリ2 (platanus_b の iterate ステップ)__

実行には数10分程度かかる。

In [ ]:
platanus_b iterate -t 2 -c out_contig.fa -IP1 ../reads/DRR024501_1.sampled.fastp.fastq ../reads/DRR024501_2.sampled.fastp.fastq

アセンブリ結果 (out_iterativeAssembly.fa) の確認

In [ ]:
seqkit stats -a -G N out_iterativeAssembly.fa

## アノテーション

本文ではアセンブリ結果ファイル (out_iterativeAssembly.fa) を [DFAST ウェブサービス](https://dfast.nig.ac.jp) にアップロードしてアノテーションを行っています。  
本文執筆後、DFAST のローカル版が bioconda でインストールできるようになったため、下記ではローカル版のインストールおよび利用方法を紹介します。


### DFAST ローカル版のインストール

インストール時に参照データベースファイルの取得等を行うため、ネットワーク環境によっては数分〜10分程度かかります。

In [ ]:
conda install -y -c bioconda dfast

動作確認

In [ ]:
dfast --version

### DFASTの実行

初回実行時には参照データベースファイルの indexing を行うため時間がかかる。二回目以降は実行時間が短縮される。

In [ ]:
dfast --genome out_iterativeAssembly.fa

結果ファイルは `OUT` ディレクトリ内に出力される。出力ファイルには、
- genome.gbk (GenBank 形式のアノテーション結果ファイル)
- genome.gff (GFF 形式のアノテーション結果ファイル)
- protein.faa (FASTA 形式のアノテーションされた遺伝子のアミノ酸配列ファイル)
- cds.fna (FASTA 形式のアノテーションされた遺伝子の CDS 塩基配列ファイル)

などが含まれる。

詳細な使用方法はヘルプ `dfast -h` や、[公式サイト](https://github.com/nigyta/dfast_core)のドキュメントをご参照ください。